#CHECK OLLAMA 

If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:

# Show available Ollama models
!ollama list

NAME                                                   ID              SIZE      MODIFIED       
llama3:latest                                          365c0bd3c000    4.7 GB    6 minutes ago     
qwen3-coder:480b-cloud                                 e30e45586389    -         20 minutes ago    
gpt-oss:120b-cloud                                     569662207105    -         20 minutes ago    
embeddinggemma:latest                                  85462619ee72    621 MB    2 weeks ago       
gpt-oss:20b                                            aa4295ac10c3    13 GB     2 weeks ago       
qwen3:30b                                              e50831eb2d91    18 GB     7 weeks ago       
hf.co/unsloth/DeepSeek-R1-0528-Qwen3-8B-GGUF:Q4_K_M    9acac7df68e1    5.0 GB    4 months ago      


In [3]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "gpt-oss:20b"

In [4]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [5]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull gpt-oss:20b\


In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

### Generative AI in Business: A Quick‑Reference Overview

| # | Business Domain | How Generative AI Helps | Typical Outputs | Why It Matters |
|---|-----------------|------------------------|-----------------|----------------|
| 1 | **Marketing & Content** | Create copy, headlines, blog posts, ad creatives, and social media snippets at scale. | Text (ads, emails, scripts), images, short videos, brand‑aligned copy. | Cuts production time, ensures consistency, frees human writers for higher‑value strategy. |
| 2 | **Customer Support & Chatbots** | Generate context‑aware replies, FAQ updates, and proactive assistance. | Natural‑language responses, knowledge‑base articles, guided troubleshooting. | 24/7 coverage, reduces ticket volume, improves CSAT. |
| 3 | **Product Design & Prototyping** | Generate new product concepts, mock‑ups, or 3D CAD files. | Design sketches, generative‑design CAD models, material‑optimized parts. | Speeds iteration, uncovers novel shapes, reduces prototyping cos

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

**Generative AI in the Enterprise: A Quick‑Guide to the Most Impactful Business Applications**

Generative AI—models that can create text, images, audio, code, or even synthetic data—has moved beyond the novelty stage and is now a catalyst for new value propositions across the entire corporate value chain. Below is a structured overview of the key application spaces, concrete use‑cases, and the business benefits they deliver.

---

## 1. **Marketing & Brand Experience**

| Use‑Case | What it Does | Typical Business Value |
|----------|--------------|------------------------|
| **Dynamic Content Generation** | Auto‑writes product descriptions, ad copy, social‑media posts, email subject lines, and landing‑page copy. | Faster go‑to‑market, higher conversion rates, consistency at scale. |
| **Personalized Storytelling** | Generates customer‑specific narratives (e.g., “Why you should choose our product” tailored to browsing history). | Increased engagement, higher CLV (customer lifetime val

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW LETS CONECT IT WITH THE INTERNET 



In [14]:

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re

# Setup OpenAI-compatible client for Ollama (assuming it's not defined earlier)
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [15]:
# Step 1: Fetch and clean the website content
def fetch_website_content(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Remove script and style elements
        for script in soup(["script", "style", "nav", "footer", "header"]):
            script.decompose()
        
        # Get text content while preserving some structure for better summarization
        text_parts = []
        for element in soup.find_all(['p', 'h1', 'h2', 'h3', 'li', 'div'], recursive=True):
            text = element.get_text(strip=True)
            if text and len(text) > 10:  # Only include meaningful text chunks
                text_parts.append(text)
        text = '\n\n'.join(text_parts)
        
        # Clean up whitespace and newlines
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        # Limit to first 2000 characters to avoid overwhelming the model
        return text[:2000]
    
    except Exception as e:
        print(f"Error fetching website: {e}")
        return None

In [16]:

# Step 2: Summarize using GPT-OSS:20b via Ollama
def summarize_with_gptoss(content, url):
    if not content:
        return "Could not fetch website content."
    
    # Create prompt for summarization
    prompt = f"""You are a helpful assistant that summarizes websites. Please provide a concise summary of the following website content from {url}:

{content}

Please provide:
1. A brief overview (2-3 sentences)
2. Key topics or main sections covered
3. The purpose or main goal of the website

Keep the summary under 200 words."""

    messages = [{"role": "user", "content": prompt}]
    
    try:
        # Using OpenAI-compatible client for Ollama with gpt-oss:20b
        response = ollama_via_openai.chat.completions.create(
            model="gpt-oss:20b",
            messages=messages,
            temperature=0.3,
            max_tokens=500
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"Error with model 'gpt-oss:20b': {e}")
        print("Please pull the model using 'ollama pull gpt-oss:20b' to enable summarization.")
        
        # Fallback to a basic rule-based summary
        lines = content.split('\n')
        first_few_lines = '\n'.join(lines[:5])
        return f"Basic summary of website content from {url}:\n\nThe website appears to contain text starting with: {first_few_lines}\n\nPlease pull the gpt-oss:20b model using 'ollama pull gpt-oss:20b' to enable full summarization."


In [17]:

# Step 3: Main function to summarize a website
def summarize_website(url):
    print(f"Summarizing website: {url}")
    content = fetch_website_content(url)
    summary = summarize_with_gptoss(content, url)
    return summary

# Example usage
if __name__ == "__main__":
    # Test with a sample website
    test_url = "https://polluxmotors.com/"  # Replace with any website URL
    summary = summarize_website(test_url)
    print("\n" + "="*50)
    print("SUMMARY:")
    print("="*50)
    print(summary)


Summarizing website: https://polluxmotors.com/

SUMMARY:
**1. Overview**  
Pollux Motors presents itself as a premium automotive brand, aiming to deliver a “luxury automotive experience.” The site showcases its vehicle lineup, design philosophy, and service offerings, though the current React‑based front end is not loading properly.

**2. Key Topics / Main Sections**  
- **Home / Hero** – Brand introduction


In [ ]:
# 🛍️ AI-POWERED SMART PRODUCT ADVISOR
# Clean, fast, and reliable product research using local LLM

import time
from IPython.display import display, Markdown

class SmartProductAdvisor:
    """
    AI-Powered Product Research Assistant using Ollama
    No web scraping needed - pure AI intelligence!
    """
    
    def __init__(self, model="gpt-oss:20b"):
        self.model = model
        self.client = ollama_via_openai
    
    def research_products(self, query, budget=None, requirements=None, num_products=5):
        """
        Generate comprehensive product research using AI
        
        Args:
            query: Product category (e.g., "wireless headphones")
            budget: Budget constraint (e.g., "under $200")
            requirements: Special needs (e.g., "for gaming")
            num_products: Number of products to recommend
        """
        # Build comprehensive search context
        context = f"Product Category: {query}"
        if budget:
            context += f"\nBudget: {budget}"
        if requirements:
            context += f"\nSpecial Requirements: {requirements}"
        
        prompt = f"""You are an expert product researcher and shopping advisor. Provide a comprehensive buying guide for:

{context}

Generate a detailed guide with the following structure:

## 🎯 PRODUCT OVERVIEW
Brief introduction to the product category and what buyers should know.

## 🏆 TOP {num_products} RECOMMENDED PRODUCTS
For each product, provide:
**{num_products}. [Brand + Model Name]** - $[Price]
- ⭐ Rating: [X.X/5.0]
- 🔑 Key Features: [List 3-4 standout features]
- ✅ Pros: [2-3 advantages]
- ❌ Cons: [1-2 disadvantages]
- 👤 Best For: [Target user profile]

## 💰 PRICE BREAKDOWN
- Budget Range ($X-$Y): [Description]
- Mid-Range ($X-$Y): [Description]  
- Premium ($X+): [Description]

## 🛒 BUYING GUIDE
### What to Look For:
- [Key feature 1 and why it matters]
- [Key feature 2 and why it matters]
- [Key feature 3 and why it matters]

### Common Mistakes to Avoid:
- [Mistake 1]
- [Mistake 2]

## 🎖️ FINAL VERDICT
- **🥇 Best Overall**: [Product name] - [One sentence why]
- **💵 Best Value**: [Product name] - [One sentence why]
- **✨ Best Premium**: [Product name] - [One sentence why]

Make recommendations realistic, specific, and based on actual 2024 market knowledge."""

        print(f"🔍 Researching: {query}")
        if budget:
            print(f"💰 Budget: {budget}")
        if requirements:
            print(f"📋 Requirements: {requirements}")
        print(f"⏳ Generating comprehensive guide with {self.model}...\n")
        
        try:
            start_time = time.time()
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=2000
            )
            
            elapsed = time.time() - start_time
            guide = response.choices[0].message.content
            
            print(f"✅ Research completed in {elapsed:.1f} seconds\n")
            print("=" * 70)
            
            return {
                "query": query,
                "budget": budget,
                "requirements": requirements,
                "guide": guide,
                "model": self.model,
                "generation_time": elapsed,
                "timestamp": time.strftime("%Y-%m-%d %H:%M:%S")
            }
            
        except Exception as e:
            print(f"❌ Error: {e}")
            print(f"💡 Make sure {self.model} is installed: ollama pull {self.model}")
            return None
    
    def display_guide(self, result):
        """Display the guide in formatted Markdown"""
        if result:
            display(Markdown(result['guide']))
            print("\n" + "=" * 70)
            print(f"📊 Generated by: {result['model']}")
            print(f"⏱️  Generation time: {result['generation_time']:.1f}s")
            print(f"🕐 Timestamp: {result['timestamp']}")
        else:
            print("No results to display")
    
    def compare_products(self, product1, product2):
        """
        Direct comparison between two specific products
        """
        prompt = f"""Compare these two products in detail:

Product A: {product1}
Product B: {product2}

Provide a comprehensive comparison with:

## 📊 SPECIFICATIONS COMPARISON
| Feature | {product1} | {product2} |
|---------|------------|------------|
[Fill in key specs side by side]

## ⚔️ HEAD-TO-HEAD ANALYSIS
### Performance
[Compare performance aspects]

### Value for Money
[Compare value proposition]

### Build Quality & Design
[Compare quality and design]

### User Experience
[Compare usability]

## 🏁 THE VERDICT
**Winner**: [Product name]
**Reason**: [Detailed explanation]

**When to choose {product1}**: [Scenarios]
**When to choose {product2}**: [Scenarios]

Be objective, detailed, and help the buyer make an informed decision."""

        print(f"⚔️  Comparing: {product1} vs {product2}")
        print(f"⏳ Analyzing with {self.model}...\n")
        
        try:
            start_time = time.time()
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
                max_tokens=1500
            )
            
            elapsed = time.time() - start_time
            comparison = response.choices[0].message.content
            
            print(f"✅ Comparison completed in {elapsed:.1f} seconds\n")
            print("=" * 70)
            
            display(Markdown(comparison))
            
            return comparison
            
        except Exception as e:
            print(f"❌ Error: {e}")
            return None



In [ ]:

# Initialize the advisor
advisor = SmartProductAdvisor(model="gpt-oss:20b")

# ============================================================================
# DEMO: Multiple Product Research Examples
# ============================================================================

print("🚀 SMART PRODUCT ADVISOR - AI-POWERED SHOPPING ASSISTANT")
print("=" * 70)
print("✨ Features: Fast, reliable, comprehensive product research")
print("🤖 Powered by: Local Ollama LLM (No web scraping needed!)")
print("=" * 70)
print("\n")

# Example 1: Wireless Headphones
print("📱 EXAMPLE 1: Wireless Headphones")
print("-" * 70)
result1 = advisor.research_products(
    query="wireless bluetooth headphones",
    budget="under $300",
    requirements="noise cancellation for travel",
    num_products=5
)
if result1:
    advisor.display_guide(result1)

print("\n\n")

# Example 2: Gaming Laptop  
print("💻 EXAMPLE 2: Gaming Laptop for Students")
print("-" * 70)
result2 = advisor.research_products(
    query="gaming laptop",
    budget="$800-$1200",
    requirements="good battery life for college student",
    num_products=5
)
if result2:
    advisor.display_guide(result2)

print("\n\n")

# Example 3: Coffee Maker
print("☕ EXAMPLE 3: Home Coffee Equipment")
print("-" * 70)
result3 = advisor.research_products(
    query="espresso machine",
    budget="$200-$600",
    requirements="easy to use for beginners",
    num_products=4
)
if result3:
    advisor.display_guide(result3)

print("\n" + "=" * 70)
print("✅ ALL EXAMPLES COMPLETED!")
print("=" * 70)


🚀 SMART PRODUCT ADVISOR - AI-POWERED SHOPPING ASSISTANT
✨ Features: Fast, reliable, comprehensive product research
🤖 Powered by: Local Ollama LLM (No web scraping needed!)


📱 EXAMPLE 1: Wireless Headphones
----------------------------------------------------------------------
🔍 Researching: wireless bluetooth headphones
💰 Budget: under $300
📋 Requirements: noise cancellation for travel
⏳ Generating comprehensive guide with gpt-oss:20b...

✅ Research completed in 59.7 seconds



## 🎯 PRODUCT OVERVIEW  
Wireless Bluetooth headphones have become the go‑to gear for travelers, commuters, and anyone who wants high‑quality sound without the hassle of wires. When you’re looking for a pair that can keep the world at bay, active noise‑cancellation (ANC) is a must‑have. Under $300, you can still find models that combine solid ANC, long battery life, and travel‑friendly ergonomics. The key is to balance price, performance, and comfort so you can fly, commute, or relax in peace.

---

## 🏆 TOP 5 RECOMMENDED PRODUCTS  

| # | Brand + Model | Price | ⭐ Rating | 🔑 Key Features | ✅ Pros | ❌ Cons | 👤 Best For |
|---|---------------|-------|----------|-----------------|--------|--------|------------|
| **1** | **Sony WH‑1000XM4** | **$199** | 4.7/5.0 | • Industry‑leading ANC<br>• Adaptive Sound Control<br>• 30 h battery (ANC on)<br>• Quick‑connect & multipoint | • Best overall ANC<br>• Comfortable over‑ear design<br>• Smart gesture controls | • Slightly heavier than some competitors | • Frequent travelers who value top‑tier ANC |
| **2** | **Bose QuietComfort 45** | **$299.99** | 4.6/5.0 | • Bose “Quiet‑Comfort” ANC<br>• 24 h battery (ANC on)<br>• Lightweight (280 g)<br>• Dual‑mode mic for calls | • Ultra‑light & breathable<br>• Excellent call quality<br>• Easy to use | • ANC slightly less aggressive than Sony | • Jet‑setters who want a lightweight, all‑day pair |
| **3** | **Sennheiser Momentum 4 Wireless** | **$280** | 4.5/5.0 | • Adaptive ANC<br>• 60 h battery (standby)<br>• Premium leather‑trimmed ear cups<br>• Touch controls & voice assistant | • Rich, detailed sound<br>• Stylish, premium build<br>• Long battery life | • Heavier (~300 g) | • Audiophiles who also travel |
| **4** | **Jabra Elite 85h** | **$199** | 4.4/5.0 | • Smart ANC with 5 microphones<br>• 36 h battery (ANC on)<br>• Weather‑resistant design<br>• SmartSound adaptive EQ | • Best battery life in class<br>• Durable & water‑resistant<br>• Good call quality | • ANC not as punchy as Sony/Bose | • Budget‑conscious travelers who need long battery |
| **5** | **Audio‑Technica ATH‑ANC700BT** | **$199** | 4.3/5.0 | • 3‑stage ANC<br>• 30 h battery (ANC on)<br>• Lightweight (220 g)<br>• Dual‑mode mic & voice assistant | • Solid ANC for price<br>• Comfortable fit<br>• Easy pairing | • Sound signature is more neutral, less “warm” | • Casual travelers who want solid ANC without premium price |

> **Tip:** Prices fluctuate with sales, so keep an eye on seasonal deals (Black Friday, Cyber Monday, Amazon Prime Day) to snag the best price.

---

## 💰 PRICE BREAKDOWN  

| Budget Range | Typical Price | What You Get |
|--------------|---------------|--------------|
| **Budget ($0–$150)** | $99–$149 | Basic ANC, decent battery, lighter build. Great for occasional use. |
| **Mid‑Range ($150–$250)** | $150–$250 | Strong ANC, longer battery, better comfort. Ideal for frequent travelers. |
| **Premium ($250+)** | $250–$300 | Top‑tier ANC, premium materials, extra features (e.g., voice assistants, multipoint). Best for audiophiles and heavy users. |

---

## 🛒 BUYING GUIDE  

### What to Look For  

1. **ANC Quality**  
   - *Why it matters:* The core of a travel‑friendly pair. Look for models with adaptive ANC that can adjust to cabin noise or city traffic.  
   - *Red flag:* “Passive” noise isolation only – it blocks sound physically but doesn’t cancel it.

2. **Battery Life & Charging**  
   - *Why it matters:* Long flights or long commutes need at least 20 h of ANC‑on battery. Quick‑charge (15‑min to 3‑h) is a plus.  
   - *Red flag:* Battery that only lasts 5–7 h with ANC on.

3. **Comfort & Build**  
   - *Why it matters:* You’ll wear them for hours. Look for padded ear cups, adjustable headband, and a weight under 300 g for travel.  
   - *Red flag:* Heavy, rigid headbands that cause neck strain.

4. **Connectivity & Controls**  
   - *Why it matters:* Multipoint pairing (connect to phone & laptop), Bluetooth 5.0+, and intuitive touch or button controls reduce hassle.  
   - *Red flag:* No multipoint support or clunky button layout.

5. **Durability & Portability**  
   - *Why it matters:* Travel exposes gear to bumps, humidity, and temperature changes. Weather‑resistant or at least sweat‑proof is a bonus.  
   - *Red flag:* No protective case or flimsy plastic parts.

### Common Mistakes to Avoid  

- **Skipping Firmware Updates**  
  - Many ANC improvements come via firmware. Forgetting to update can leave you with sub‑optimal noise cancellation.

- **Ignoring Fit & Ear Cup Size**  
  - A pair that feels great on a showroom mannequin can be uncomfortable on your head. Try them on, or read detailed fit reviews.

- **Overlooking Battery Health**  
  - Batteries degrade over time. If buying used or refurbished, ask about


📊 Generated by: gpt-oss:20b
⏱️  Generation time: 59.7s
🕐 Timestamp: 2025-10-01 17:28:49



💻 EXAMPLE 2: Gaming Laptop for Students
----------------------------------------------------------------------
🔍 Researching: gaming laptop
💰 Budget: $800-$1200
📋 Requirements: good battery life for college student
⏳ Generating comprehensive guide with gpt-oss:20b...



In [ ]:
# 🎮 CUSTOM USAGE EXAMPLES
# Use the advisor for your own product research!

# Example 1: Simple product research
print("=" * 70)
print("CUSTOM EXAMPLE 1: Quick Product Research")
print("=" * 70)

smartphone_guide = advisor.research_products(
    query="flagship smartphone",
    budget="under $900",
    requirements="best camera quality",
    num_products=4
)
advisor.display_guide(smartphone_guide)

print("\n\n")

# Example 2: Product comparison
print("=" * 70)
print("CUSTOM EXAMPLE 2: Direct Product Comparison")
print("=" * 70)

comparison = advisor.compare_products(
    product1="Sony WH-1000XM5",
    product2="Bose QuietComfort Ultra"
)

print("\n" + "=" * 70)
print("💡 TIP: Customize these examples for any product you want to research!")
print("=" * 70)
